## Code for analyzing the Today-Asahi dataset
#### Data Management (Spring/Summer 2018) at OSIPP, Osaka U

### R version

### Preamble

In [107]:
library(dplyr)
library(data.table) 
library(magrittr)
library(stargazer) # install.packages("stargazer", repos='http://cran.us.r-project.org')
library(ggplot2)
library(repr)
library(lfe) # install.packages("lfe", repos='http://cran.us.r-project.org')
library(plm) # install.packages("plm", repos='http://cran.us.r-project.org')

In [108]:
setwd("..") # set the parent directory as a working directory. Alternatively, specify the path to your local directory.

In [109]:
options(repr.plot.width=4, repr.plot.height=4) # set figure size

### Import data

In [ ]:
ta_panel <- fread('build_input/todai-asahi/output/syuuin_2009_2014_R.csv')
print(dim(ta_panel))
ta_panel <- subset(ta_panel,(PREFEC != 66) & (!is.na(PREFEC))) # remove PR and missing prefec
print(dim(ta_panel))

### Check contents

In [ ]:
print(head(ta_panel)) # top 5 rows

In [ ]:
print(str(ta_panel)) # get structure 

In [ ]:
print(summary(ta_panel)) # get a summary table

In [ ]:
print("total number of NA")
print(apply(is.na(ta_panel),2,sum)) 
print("total number of NaN")
print(apply(is.nan(as.matrix(ta_panel)),2,sum)) 
print("total number of inf or -inf")
print(apply(is.infinite(as.matrix(ta_panel)),2,sum)) 

### Make a summary table

In [ ]:
stargazer(ta_panel,out="analysis_output/sum_stat_R.tex")

#### - Group summary
- Use `group_by()`.

In [ ]:
ta_panel2 <- subset(ta_panel,yn_fiscalpol != 99)
by_set <- group_by(ta_panel2,SEX)
summarize(by_sex,count=n(),mean=mean(yn_fiscalpol,na.rm=TRUE))

- Useful to use pipe operation.
- `x %>% f(y)` becomes `f(x, y)`. `x %>% f(y) %>% g(z)` becomes `g(f(x, y), z)`.  

In [ ]:
ta_panel %>%
    subset(yn_fiscalpol != 99) %>%
    group_by(SEX) %>%
    summarize(
        count = n(),
        mean = mean(yn_fiscalpol,na.rm = TRUE))

### Make figures

#### - Histograms

In [ ]:
ta_panel2 <- subset(ta_panel,yn_fiscalpol != 99)

ggplot(data=ta_panel2,aes(yn_fiscalpol)) + 
    geom_histogram(binwidth=1,alpha=0.3,fill="black") +
    scale_y_continuous(breaks=seq(0,1250,by=200)) +
    labs(title="Title",x="Fiscal policy",y="") +
    theme(
        panel.background = element_rect(fill=NA),
        panel.border = element_rect(fill=NA,color="grey75"),
        axis.ticks = element_line(color="grey85"),
        #panel.grid.major = element_line(color = "grey95", size = 0.2),
        #panel.grid.minor = element_line(color = "grey95", size = 0.2),
        legend.position = "none",
        plot.title = element_text(hjust=0.5,size=9),
        axis.title = element_text(size=9),
        axis.text = element_text(size=9)
    )
ggsave("analysis_output/yn_fiscalpol_R.png",width=4,height=4)

In [ ]:
ta_panel2 <- subset(ta_panel,yn_fiscalpol != 99)
ta_panel2$SEX <- factor(ta_panel2$SEX)
levels(ta_panel2$SEX) <- c("Male","Female")

ggplot(data=ta_panel2,aes(yn_fiscalpol,fill=factor(SEX))) +
    geom_histogram(binwidth=1,alpha=0.3) +
    scale_y_continuous(breaks=seq(0,1250,by=200)) +
    facet_grid(. ~ SEX) +
    scale_fill_manual(values=c("blue","red")) + 
    labs(title="Title",x="Fiscal policy",y="") +
    theme(
        panel.background = element_rect(fill=NA),
        panel.border = element_rect(fill=NA,color="grey75"),
        axis.ticks = element_line(color="grey85"),
        #panel.grid.major = element_line(color = "grey95", size = 0.2),
        #panel.grid.minor = element_line(color = "grey95", size = 0.2),
        legend.position = "none",
        plot.title = element_text(hjust=0.5,size=9),
        axis.title = element_text(size=9),
        axis.text = element_text(size=9)
    )
ggsave("analysis_output/yn_fiscalpol_bysex_R.png",width=4,height=4)

In [ ]:
ta_panel2 <- subset(ta_panel,yn_fiscalpol != 99)
ta_panel2$SEX <- factor(ta_panel2$SEX)
levels(ta_panel2$SEX) <- c("Male","Female")

ggplot(data=ta_panel2,aes(yn_fiscalpol,fill=SEX))+
    geom_histogram(binwidth=1,alpha=0.3,position='identity') +
    scale_y_continuous(breaks=seq(0,1250,by=200)) +
    scale_fill_manual(values=c("blue","red")) + 
    labs(title="Title",x="Fiscal policy",y="") +
    theme(
        panel.background = element_rect(fill=NA),
        panel.border = element_rect(fill=NA,color="grey75"),
        axis.ticks = element_line(color="grey85"),
        #panel.grid.major = element_line(color = "grey95", size = 0.2),
        #panel.grid.minor = element_line(color = "grey95", size = 0.2),
        legend.position = "none",
        plot.title = element_text(hjust=0.5,size=9),
        axis.title = element_text(size=9),
        axis.text = element_text(size=9)
    )

#### - Kernel density plots

In [ ]:
set.seed(123456789)
normal <- data.frame(value=rnorm(1000,0,1))
ggplot(data=normal,aes(x=value)) +
    geom_density(aes(color='red',alpha=0.3)) +
    geom_histogram(bins=100,alpha=0.3,aes(y=..density..)) +
    labs(title="Title",x="",y="") +
    theme(
        panel.background = element_rect(fill=NA),
        panel.border = element_rect(fill=NA,color="grey75"),
        axis.ticks = element_line(color="grey85"),
        #panel.grid.major = element_line(color = "grey95", size = 0.2),
        #panel.grid.minor = element_line(color = "grey95", size = 0.2),
        legend.position = "none",
        plot.title = element_text(hjust=0.5,size=9),
        axis.title = element_text(size=9),
        axis.text = element_text(size=9)
    )

#### - Scatter plots

In [ ]:
ta_panel2 <- subset(ta_panel,(fav_ozawa != 999) & (!is.na(AGE)))

ggplot(data=ta_panel2,aes(x=fav_ozawa,y=AGE))+
    geom_point(color='blue') +
    labs(title="Title",x="fav_ozawa",y="Age") +
    theme(
        panel.background = element_rect(fill = NA),
        panel.border = element_rect(fill = NA, color = "grey75"),
        axis.ticks = element_line(color = "grey85"),
        #panel.grid.major = element_line(color = "grey95", size = 0.2),
        #panel.grid.minor = element_line(color = "grey95", size = 0.2),
        legend.position = "none",
        plot.title = element_text(hjust = 0.5, size=9),
        axis.title = element_text(size=9),
        axis.text = element_text(size=9)
    )

#### - Line plots

In [ ]:
set.seed(123456789)
n <- 1000

wn <- data.frame(
    sample = 1:n,
    group=rep(c("one","two","three"),each=n), 
        value=c(cumsum(rnorm(n,0,1)), 
                cumsum(rnorm(n,0,1)),
                cumsum(rnorm(n,0,1)))
    )

ggplot(data=wn,mapping=aes(x=sample,y=value)) +
    geom_line(mapping=aes(colour=group)) +
    labs(title="Title", x="", y="") +
    scale_color_manual(name="",values=c("blue","red","yellow")) +
    theme(
        panel.background = element_rect(fill=NA),
        panel.border = element_rect(fill=NA, color="grey75"),
        axis.ticks = element_line(color="grey85"),
        #panel.grid.major = element_line(color = "grey95", size = 0.2),
        #panel.grid.minor = element_line(color = "grey95", size = 0.2),
        legend.position = "bottom",
        legend.key = element_blank(),
        legend.text = element_text(size=8),
        plot.title = element_text(hjust = 0.5, size=9),
        axis.title = element_text(size=9),
        axis.text = element_text(size=9)
    )

### Make regression tables

In [ ]:
ta_panel2 <- subset(ta_panel,ab_asiaus != 99)

reg1 <- lm(ab_asiaus ~ AGE,data=ta_panel2)
reg2 <- lm(ab_asiaus ~ AGE + PREFEC,data=ta_panel2)

stargazer(reg1,reg2,title="OLS Regressions",align=TRUE,type="text")
stargazer(reg1,reg2,title="OLS Regressions",align=TRUE,out="analysis_output/ols_results_R.tex")

#### - Include dummies

In [ ]:
ta_panel2 <- subset(ta_panel,(yn_nkorea != 99) & (yn_preemp != 99) & (is.na(yn_nkorea) == FALSE) & (is.na(yn_preemp) == FALSE))

reg1 <- lm(yn_preemp ~ yn_nkorea,data=ta_panel2)
reg2 <- felm(yn_preemp ~ yn_nkorea | uid + ELECYEAR,data=ta_panel2)

stargazer(reg1,reg2,title="OLS Regressions",align=TRUE,type = "text")

- Use `plm` (not recommended).

In [ ]:
ta <- pdata.frame(ta_panel,index=c("uid","ELECYEAR"),drop.index=TRUE,row.names=TRUE)
ta2 <- subset(ta,yn_nkorea != 99 & yn_preemp != 99)

reg <- plm(yn_preemp ~ yn_nkorea,data=ta2, model="within",effect="twoway")

stargazer(reg,title="OLS Regressions",align=TRUE,type="text")

- Cluster standard errors.

In [ ]:
ta_panel2 <- subset(ta_panel, yn_nkorea != 99 & yn_preemp != 99)

reg1 <- felm(yn_preemp ~ yn_nkorea, data=ta_panel2)
reg2 <- felm(yn_preemp ~ yn_nkorea | 0 | 0 | uid, data=ta_panel2)

stargazer(reg1, reg2, title="OLS Regressions", align=TRUE, type = "text")